In [2]:
import sys
import os
import torch
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import  open_json
from utils.util import is_target_category

import json
import glob
import random

from utils.util import get_category
from utils.util import calc_compatibility_score
from utils.util import get_progress_percent
import tomotopy as tp
from itertools import chain


In [3]:
import importlib
import utils
importlib.reload(utils.util)
from utils.util import calc_cw_score
from utils.util import scale_versatility, scale_compatibility
from utils.infer import topic_model_infer

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [5]:
filepaths = glob.glob(
    f"C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
)

In [6]:
id_to_vector_compatibility = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_compatibility.json')

In [7]:
items = {"tops": [], "bottoms": [], "shoes": []}
for fp in filepaths:
    json_dict = open_json(fp)
    if json_dict == None:
        continue
    for item in list(filter(is_target_category, json_dict["items"])):
        category = get_category(item)
        if category == None:
            continue
        items[category].append(str(item['itemId']))

In [8]:
categories = ['tops', 'bottoms', 'shoes']
# 枚数
T = 3

In [9]:
mdl = tp.LDAModel.load(
    "C:/Users/yuuta/Documents/fashion/model_learning/topic_model/models/lda.bin"
)
with open(
    "C:/Users/yuuta/Documents/fashion/data/attributes_test.json",
    encoding="shift-jis"
) as f:
    to_attributes = json.load(f)

In [10]:
# (cw, proposal_score, topic_model_score)の配列
com_score = []
dataset = {
    'tops':items["tops"],
    'bottoms':items["bottoms"],
    'shoes':items["shoes"],
}
N = 100000
for i in range(N):
    if i % 100 == 0:
        print(get_progress_percent(i, N), end='')
    
    cw = {
        'tops':[random.choice(dataset['tops'])],
        'bottoms':[random.choice(dataset['bottoms'])],
        'shoes':[random.choice(dataset['shoes'])],
    }
    pc = calc_compatibility_score(cw, id_to_vector_compatibility)
    attributes = []
    for itemId in list(chain(*list(cw.values()))):
        attr = to_attributes[itemId]
        attributes += attr
    tc = topic_model_infer(mdl, attributes)
    com_score.append((cw, pc, tc))

【******************* 】

In [11]:
proposed_ranking = sorted(com_score, key=lambda x: x[1], reverse=True)
topic_model_ranking = sorted(com_score, key=lambda x: x[2], reverse=True)

In [12]:
proposed_ranking[0], proposed_ranking[-1]

(({'tops': ['39821188'], 'bottoms': ['19968934'], 'shoes': ['29921333']},
  0.01596080814986102,
  -41.20652770996094),
 ({'tops': ['9312811'], 'bottoms': ['9465634'], 'shoes': ['38787338']},
  0.0015566368700805824,
  -46.337181091308594))

In [14]:
import importlib
import utils
# importlib.reload(utils.save_image)

from utils.save_image import save_combined_image


In [15]:
def save_image(ranking, rank, path):
    item_ids = list(chain(*list(ranking[rank][0].values())))
    img = save_combined_image(item_ids, f'C:/Users/yuuta/Documents/fashion/experiments/questionnaire/images/compatibility/{path}', W=1)
    return img

In [17]:
for rank in range(1, N, 5000):
    save_image(proposed_ranking, rank, f'proposed/{rank}.png')
    save_image(topic_model_ranking, rank, f'lda/{rank}.png')

1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
1 3 size: (150, 450)
